# DL Survival - Ventilation Outcomes
 Updated 21/11/21

In [266]:
import pandas as pd
import numpy as np
import math
import statistics
from datetime import datetime
import datetime as dt
from datetime import timedelta
import json
import miceforest as mf

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

pd.set_option('display.max_columns', None)

## 1. Data cleaning

- Import MIMIC III data
- Review column unique values, assign correct data types
- Impute missing values


### 1.1: Importing data

In [267]:
df = pd.read_csv('mimic_combined.csv')
print(df.shape)
df.head(10)

(12332, 110)


,Unnamed: 0,hospital_expire_flag,los,spo2,free_calcium,outtime,meanbp,ptt,tidalvol,wcc,cvd,weight,bicarb,ggt,t1dm,temp,malig,subject_id,hr,baseexcess,diab_un,first_careunit,hadm_id,bilirubin_direct,liver_severe,ventrate,fibrinogen,arrhythmia,neutrophils,prbc,glucose,magnesium,po2,ext_time,alp,tricuspid,albumin,dementia,dischtime,pulmonary,t2dm,plts,lactate,bleed_time,admission_location,rr,mit,insulin,pvd,pud,lymphocytes,gender,cabg,smoking,reintubation,height,inr,bilirubin_total,diab_cc,creatinine,insurance,mi,specimen,deathtime,pt,aado2,hba1c,crp,pco2,aids,language,dod,dbp,reint_time,intime,rheum,bg_temp,sbp,chloride,fio2,sodium,last_careunit,infection,paraplegia,cardiac_index,marital_status,potassium,bilirubin_indirect,bun,dtoutput,ckd,copd,cryo,admission_type,met_ca,hb,ethnicity,admittime,ffp,inr_1,ccf,icustay_seq,ph,ast,alt,plt,aortic,vent_array,hematocrit,liver_mild
0,0,0,2.2769,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",2198-02-02 19:06:39,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",[],"[{'charttime': datetime.datetime(2198, 1, 31, ...",0,84.00,"[{'charttime': datetime.datetime(2198, 1, 31, ...",[],0,"[{'charttime': datetime.datetime(2198, 1, 31, ...",0,27328,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",0,CSRU,195663,[],0,[],[],0,[],[],"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 2, 2, 0...","[{'charttime': datetime.datetime(2198, 1, 31, ...",2198-01-31 22:00:00,[],0,[],0,2198-02-04 12:00:00,0,0,[],"[{'charttime': datetime.datetime(2198, 1, 31, ...",[],PHYS REFERRAL/NORMAL DELI,"[{'charttime': datetime.datetime(2198, 1, 31, ...",0,[],0,0,[],M,1,0,0,172.72,"[{'charttime': datetime.datetime(2198, 1, 31, ...",[],0,"[{'charttime': datetime.datetime(2198, 1, 31, ...",Private,1,[],NaN,[],[],[],[],"[{'charttime': datetime.datetime(2198, 1, 31, ...",0,ENGL,NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,2198-01-31 12:27:58,0,[],"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",[],"[{'charttime': datetime.datetime(2198, 1, 31, ...",CSRU,"[{'suspected_infection_time': None, 'antibioti...",0,[],SINGLE,"[{'charttime': datetime.datetime(2198, 1, 31, ...",[],"[{'charttime': datetime.datetime(2198, 1, 31, ...",[],0,0,[],ELECTIVE,0,"[{'charttime': datetime.datetime(2198, 1, 31, ...",white,2198-01-31 08:00:00,[],"[{'charttime': datetime.datetime(2198, 1, 31, ...",0,1,"[{'charttime': datetime.datetime(2198, 1, 31, ...",[],[],"[{'charttime': datetime.datetime(2198, 1, 31, ...",0,"[{'starttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",0
1,1,0,2.2722,"[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...",2198-05-10 19:46:00,"[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...",[],"[{'charttime': datetime.datetime(2198, 5, 10, ...",0,60.00,"[{'charttime': datetime.datetime(2198, 5, 9, 3...",[],0,"[{'charttime': datetime.datetime(2198, 5, 8, 1...",0,6280,"[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...",0,CSRU,106984,[],0,[],[],0,[],"[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 9, 3...","[{'charttime': datetime.datetime(2198, 5, 8, 1...",2198-05-09 09:29:00,[],0,[],0,2198-05-15 13:49:00,0,0,"[{'charttime': datetime.datetime(2198, 5, 8, 1...",[],[],PHYS REFERRAL/NORMAL DELI,"[{'charttime': datetime.datetime(2198, 5, 8, 1...",0,"[{'charttime': datetime.datetime(2198, 5, 8, 1...",1,0,[],F,0,0,0,170.18,"[{'charttime': datetime.datetime(2198, 5, 8, 1...",[],0,"[{'charttime': datetime.datetime(2198, 5, 9, 3...",Self Pay,0,[],NaN,[],[],[],[],"[{'charttime': datetime.

#### 1.1.1: Column lists

In [268]:
#view and reorder columns
cols = list(df.columns)
new_cols = ['Unnamed: 0','hadm_id','subject_id','gender','ethnicity','marital_status','insurance','language','aortic','mit','tricuspid',
            'pulmonary','cabg','temp','bg_temp','hr','spo2','rr','sbp','dbp','meanbp','weight','height','cardiac_index','pt','ptt',
            'inr','inr_1','fibrinogen','hb','hematocrit','plts','wcc','lymphocytes','neutrophils','alp','ast','alt','ggt',
            'bilirubin_indirect','bilirubin_direct','bilirubin_total','chloride','magnesium','potassium','crp','bleed_time','albumin',
            'creatinine','free_calcium','sodium','bicarb','bun','hba1c','glucose','lactate','po2','pco2','baseexcess','ph','aado2',
            'fio2','ffp','insulin','cryo','prbc','infection','ventrate','tidalvol','vent_array','reintubation','liver_severe','liver_mild',
            'rheum','cvd','aids','ckd','copd','arrhythmia','pud','smoking','pvd','paraplegia','ccf','met_ca','t2dm','t1dm','malig','mi',
            'dementia','first_careunit','last_careunit','admission_location','admission_type','hospital_expire_flag','admittime',
            'dischtime','intime','outtime','ext_time','reint_time','los','icustay_seq','deathtime','plt','diab_un','diab_cc',
            'dtoutput','specimen','dod']

ptinfo=['Unnamed: 0','hadm_id','subject_id']

demographics=['gender','ethnicity','marital_status','insurance','language']

proceduretype=['aortic','mit','tricuspid','pulmonary','cabg']

vitals=['temp','bg_temp','hr','spo2','rr','sbp','dbp','meanbp','weight','height','cardiac_index']

labs=['pt','ptt','inr','inr_1','fibrinogen','hb','hematocrit','plts','wcc','lymphocytes','neutrophils','alp','ast','alt','ggt',
'bilirubin_indirect','bilirubin_direct','bilirubin_total','chloride','magnesium','potassium','crp','bleed_time',
'albumin','creatinine','free_calcium','sodium','bicarb','bun','hba1c','glucose','lactate']

bloodgases=['po2','pco2','baseexcess','ph','aado2','fio2']

products=['ffp','insulin','cryo','prbc','infection']

ventilation=['ventrate','tidalvol','vent_array','reintubation']

comorbidities=['liver_severe','liver_mild','rheum','cvd','aids','ckd','copd','arrhythmia','pud','smoking','pvd',
'paraplegia','ccf','met_ca','t2dm','t1dm','malig','mi','dementia']

adm_cat=['first_careunit','last_careunit','admission_location','admission_type','hospital_expire_flag']

adm_num=['admittime','dischtime','intime','outtime','ext_time','reint_time','los','icustay_seq','deathtime']

others=['plt','diab_un','diab_cc','dtoutput','specimen','dod']

timeseries=[*vitals,*labs,*bloodgases,*products,*ventilation,'plt','dtoutput']
timeseries = [i for i in timeseries if i not in ('weight','height','reintubation', 'infection', 'vent_array')]
    
timeseries_valuenames = {'cardiac_index':'ci',
                         'plts':'bloodproduct',
                         'ffp':'bloodproduct',
                         'insulin':'amount',
                         'cryo':'bloodproduct',
                         'prbc':'bloodproduct',
                         'dtoutput':'output'}

In [269]:
df = df[new_cols]
df.head(10)

,Unnamed: 0,hadm_id,subject_id,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,temp,bg_temp,hr,spo2,rr,sbp,dbp,meanbp,weight,height,cardiac_index,pt,ptt,inr,inr_1,fibrinogen,hb,hematocrit,plts,wcc,lymphocytes,neutrophils,alp,ast,alt,ggt,bilirubin_indirect,bilirubin_direct,bilirubin_total,chloride,magnesium,potassium,crp,bleed_time,albumin,creatinine,free_calcium,sodium,bicarb,bun,hba1c,glucose,lactate,po2,pco2,baseexcess,ph,aado2,fio2,ffp,insulin,cryo,prbc,infection,ventrate,tidalvol,vent_array,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,admittime,dischtime,intime,outtime,ext_time,reint_time,los,icustay_seq,deathtime,plt,diab_un,diab_cc,dtoutput,specimen,dod
0,0,195663,27328,M,white,SINGLE,Private,ENGL,0,0,0,0,1,"[{'charttime': datetime.datetime(2198, 1, 31, ...",[],"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",84.00,172.72,[],[],"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",[],"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",[],"[{'charttime': datetime.datetime(2198, 1, 31, ...",[],[],[],[],[],[],[],[],[],"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 2, 2, 0...","[{'charttime': datetime.datetime(2198, 1, 31, ...",[],[],[],"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",[],"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",[],[],[],[],[],[],"[{'suspected_infection_time': None, 'antibioti...",[],[],"[{'starttime': datetime.datetime(2198, 1, 31, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,CSRU,CSRU,PHYS REFERRAL/NORMAL DELI,ELECTIVE,0,2198-01-31 08:00:00,2198-02-04 12:00:00,2198-01-31 12:27:58,2198-02-02 19:06:39,2198-01-31 22:00:00,NaN,2.2769,1,NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...",0,0,[],[],NaN
1,1,106984,6280,F,other,DIVORCED,Self Pay,SPAN,1,0,0,0,0,"[{'charttime': datetime.datetime(2198, 5, 8, 1...",[],"[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...",60.00,170.18,"[{'charttime': datetime.datetime(2198, 5, 8, 1...",[],"[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...",[],"[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 10, ...",[],[],[],[],[],[],[],[],[],"[{'charttime': datetime.datetime(2198, 5, 9, 3...","[{'charttime': datetime.datetime(2198, 5, 9, 3...","[{'charttime': datetime.datetime(2198, 5, 8, 1...",[],[],[],"[{'charttime': datetime.datetime(2198, 5, 9, 3...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(219

### 1.2: Cleaning data types

#### 1.2.0: NaN assignment

In [270]:
df = df.replace('NaT',np.datetime64('NaT'))
df = df.replace(['[]','NaN',np.datetime64('NaT')],np.NaN)

#### 1.2.1: Datetime columns

In [271]:
# set column types as datetime
time_cols = ['admittime','dischtime','intime','outtime','reint_time','ext_time','deathtime']
for col in time_cols:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')

#dod
df['dod'] = pd.to_datetime(df['dod'], format='%Y-%m-%d')

In [272]:
## CHECK FOR ROWS WHERE DEATHTIME < INTIME OR ADMITTIME

In [273]:
df[time_cols].dtypes

admittime     datetime64[ns]
dischtime     datetime64[ns]
intime        datetime64[ns]
outtime       datetime64[ns]
reint_time    datetime64[ns]
ext_time      datetime64[ns]
deathtime     datetime64[ns]
dtype: object

#### 1.2.2: Demographics

In [274]:
for x in demographics:
    print(x,': ',df[x].unique())

gender :  ['M' 'F']
ethnicity :  ['white' 'other' 'unknown' 'hispanic' 'black' 'asian' 'native' 'WHITE'
 'UNKNOWN' 'OTHER' 'BLACK/AFRICAN AMERICAN' 'HISPANIC/LATINO'
 'UNABLE TO OBTAIN' 'ASIAN' 'AMERICAN INDIAN/ALASKA NATIVE']
marital_status :  ['SINGLE' 'DIVORCED' 'MARRIED' 'WIDOWED' 'UNKNOWN (DEFAULT)' nan
 'SEPARATED']
insurance :  ['Private' 'Self Pay' 'Medicare' 'Medicaid' 'Government' 'Other']
language :  ['ENGL' 'SPAN' nan 'VIET' 'RUSS' 'HAIT' 'CANT' 'PORT' 'PTUN' 'ALBA' 'THAI'
 'ARAB' 'GREE' 'AMER' '*LEB' '*BEN' 'CAPE' 'POLI' 'ITAL' 'HIND' 'URDU'
 'KORE' 'GERM' 'TURK' 'ETHI' 'CAMB' 'MAND' '*GUJ' 'PERS' 'ENGLISH' '?']


In [275]:
#ethnicity
df.replace({'ethnicity':
                {'unknown': np.NaN,'UNKNOWN':np.NaN,'UNABLE TO OBTAIN':np.NaN,
                'OTHER':'other','WHITE':'white','BLACK/AFRICAN AMERICAN':'black','ASIAN':'asian',
                'HISPANIC/LATINO':'hispanic','AMERICAN INDIAN/ALASKA NATIVE':'native'
                }
            }, 
            inplace=True)
print(df['ethnicity'].unique())

['white' 'other' nan 'hispanic' 'black' 'asian' 'native']


In [276]:
#marital_status
df.replace({'marital_status':
                {'UNKNOWN (DEFAULT)': np.NaN
                }
            }, 
            inplace=True)
print(df['marital_status'].unique())

['SINGLE' 'DIVORCED' 'MARRIED' 'WIDOWED' nan 'SEPARATED']


In [277]:
#language
df.replace({'language':
                {'ENGLISH':'ENGL','?':np.NaN
                }
            }, 
            inplace=True)
print(df['marital_status'].unique())

['SINGLE' 'DIVORCED' 'MARRIED' 'WIDOWED' nan 'SEPARATED']


#### 1.2.3: ✔Procedure type

In [278]:
for x in proceduretype:
    print(x,': ',df[x].unique())

aortic :  [0 1]
mit :  [0 1]
tricuspid :  [0 1]
pulmonary :  [0 1]
cabg :  [1 0]


#### 1.2.4: **Vitals / Blood Gases / Products + infection / Ventilation


In [279]:
# wait for Jahan/others
# ventrate seems to be empty

#### 1.2.5: ✔Comorbidities

In [280]:
for x in comorbidities:
    print(x,': ',df[x].unique())

liver_severe :  [0 1]
liver_mild :  [0 1]
rheum :  [0 1]
cvd :  [0 1]
aids :  [0 1]
ckd :  [0 1]
copd :  [0 1]
arrhythmia :  [0 1]
pud :  [0 1]
smoking :  [0 1]
pvd :  [0 1]
paraplegia :  [0 1]
ccf :  [0 1]
met_ca :  [0 1]
t2dm :  [0 1]
t1dm :  [0 1]
malig :  [0 1]
mi :  [1 0]
dementia :  [0 1]


#### 1.2.6: Admissions (categorical)

In [281]:
for x in adm_cat:
    print(x,': ',df[x].unique())

first_careunit :  ['CSRU' 'CCU' 'TSICU' 'SICU' 'MICU'
 'Cardiac Vascular Intensive Care Unit (CVICU)' 'Coronary Care Unit (CCU)'
 'Trauma SICU (TSICU)' 'Medical Intensive Care Unit (MICU)'
 'Medical/Surgical Intensive Care Unit (MICU/SICU)'
 'Surgical Intensive Care Unit (SICU)'
 'Neuro Surgical Intensive Care Unit (Neuro SICU)' 'Neuro Intermediate']
last_careunit :  ['CSRU' 'CCU' 'MICU' 'SICU' 'TSICU'
 'Cardiac Vascular Intensive Care Unit (CVICU)' 'Coronary Care Unit (CCU)'
 'Medical Intensive Care Unit (MICU)' 'Trauma SICU (TSICU)'
 'Medical/Surgical Intensive Care Unit (MICU/SICU)'
 'Surgical Intensive Care Unit (SICU)'
 'Neuro Surgical Intensive Care Unit (Neuro SICU)']
admission_location :  ['PHYS REFERRAL/NORMAL DELI' 'TRANSFER FROM HOSP/EXTRAM'
 'EMERGENCY ROOM ADMIT' 'CLINIC REFERRAL/PREMATURE'
 'TRANSFER FROM OTHER HEALT' 'TRANSFER FROM SKILLED NUR'
 'PHYSICIAN REFERRAL' 'TRANSFER FROM HOSPITAL' 'EMERGENCY ROOM' 'PACU'
 'PROCEDURE SITE' 'TRANSFER FROM SKILLED NURSING FACILITY

In [282]:
#first_careunit
df.replace({'first_careunit':
                {'Cardiac Vascular Intensive Care Unit (CVICU)':'CVICU',
                'Coronary Care Unit (CCU)':'CCU',
                'Medical Intensive Care Unit (MICU)':'MICU',
                'Surgical Intensive Care Unit (SICU)':'SICU',
                'Neuro Intermediate':'Neuro Inter',
                'Medical/Surgical Intensive Care Unit (MICU/SICU)':'MICU/SICU',
                'Trauma SICU (TSICU)':'TSICU',
                'Neuro Surgical Intensive Care Unit (Neuro SICU)':'Neuro SICU'
                }
            }, 
            inplace=True)
print(df['first_careunit'].unique())

['CSRU' 'CCU' 'TSICU' 'SICU' 'MICU' 'CVICU' 'MICU/SICU' 'Neuro SICU'
 'Neuro Inter']


In [283]:
#last_careunit
df.replace({'last_careunit':
                {'Cardiac Vascular Intensive Care Unit (CVICU)':'CVICU',
                'Coronary Care Unit (CCU)':'CCU',
                'Medical Intensive Care Unit (MICU)':'MICU',
                'Surgical Intensive Care Unit (SICU)':'SICU',
                'Neuro Intermediate':'Neuro Inter',
                'Medical/Surgical Intensive Care Unit (MICU/SICU)':'MICU/SICU',
                'Trauma SICU (TSICU)':'TSICU',
                'Neuro Surgical Intensive Care Unit (Neuro SICU)':'Neuro SICU'
                }
            }, 
            inplace=True)
print(df['last_careunit'].unique())

['CSRU' 'CCU' 'MICU' 'SICU' 'TSICU' 'CVICU' 'MICU/SICU' 'Neuro SICU']


In [284]:
#admission_location
df.replace({'admission_location':
                {'TRANSFER FROM HOSP/EXTRAM':'TRANSFER FROM HOSPITAL',
                'PHYS REFERRAL/NORMAL DELI':'PHYSICIAN REFERRAL',
                'TRANSFER FROM SKILLED NUR':'TRANSFER FROM SKILLED NURSING FACILITY',
                'INFORMATION NOT AVAILABLE':np.NaN,
                'CLINIC REFERRAL':'CLINIC REFERRAL/PREMATURE',
                'EMERGENCY ROOM ADMIT':'EMERGENCY ROOM',
                }
            }, 
            inplace=True)
print(df['admission_location'].unique())

['PHYSICIAN REFERRAL' 'TRANSFER FROM HOSPITAL' 'EMERGENCY ROOM'
 'CLINIC REFERRAL/PREMATURE' 'TRANSFER FROM OTHER HEALT'
 'TRANSFER FROM SKILLED NURSING FACILITY' 'PACU' 'PROCEDURE SITE'
 'WALK-IN/SELF REFERRAL' nan 'INTERNAL TRANSFER TO OR FROM PSYCH'
 'AMBULATORY SURGERY TRANSFER']


#### 1.2.7: Others

In [285]:
# for x in others:
#     print(x,': ',df[x].unique())

### Parsing time series data

In [286]:
df['vent_array'][14]

"[{'starttime': datetime.datetime(2184, 1, 18, 1, 43), 'endtime': datetime.datetime(2184, 1, 18, 4, 20), 'duration_hours': 2.6166666666666667}]"

In [287]:
def va_parser(row, output=6):
    """
    Takes row index from `df` returns a list of starttime, endtime, vent duration 
    for first and (if applicable) second intubations

    Parameters
    ----------
    row : row in df
    output_ : select which output you want (use list index below) - e.g. args=[6] for all output when using df.apply()

    Returns
    -------
    single list variable containing  
        [0] int_time1: first intubation starttime
        [1] ext_time1: first intubation endtime
        [2] duration1: first intubation duration
        [3] int_time2: second intubation starttime
        [4] ext_time2: second intubation endtime 
        [5] duration2: second intubation duration
        [6] all

    """
    int_time1=np.NaN
    ext_time1=np.NaN
    duration1=np.NaN
    int_time2=np.NaN
    ext_time2=np.NaN
    duration2=np.NaN
    value = row['vent_array']
    list=[]
    '''a = value
    print(value)'''
    if value == np.NaN or pd.isna(value):
        return np.NaN
    a = value.replace("'",'"')
    a = a.replace('\n ...\n',',').replace('\n', ',').replace('...', '')
    a = a.replace('datetime.','"dt.')
    a = a.replace('),', ')",')
    a = json.loads(a)
    b = [(i['starttime'], i['endtime'], i['duration_hours']) for i in a]
    int_time1=dt.datetime.strptime(b[0][0],'dt.datetime(%Y, %m, %d, %H, %M)')
    ext_time1=dt.datetime.strptime(b[0][1],'dt.datetime(%Y, %m, %d, %H, %M)')
    duration1=b[0][2]
    
    if output==0:
        return int_time1
    if output==1:
        return ext_time1
    if output==2:
        return duration1

    if len(b)>=2:
        int_time2=dt.datetime.strptime(b[1][0],'dt.datetime(%Y, %m, %d, %H, %M)')
        ext_time2=dt.datetime.strptime(b[1][1],'dt.datetime(%Y, %m, %d, %H, %M)')
        duration2=b[1][2]
    if output==3:
        return int_time2
    if output==4:
        return ext_time2
    if output==5:
        return duration2
    if output==6:
        return int_time1, ext_time1, duration1, int_time2, ext_time2, duration2

In [288]:
df['int_time1']=df.apply(va_parser, args=[0], axis=1)
df['ext_time1']=df.apply(va_parser, args=[1], axis=1)
df['duration1']=df.apply(va_parser, args=[2], axis=1)
df['int_time2']=df.apply(va_parser, args=[3], axis=1)
df['ext_time2']=df.apply(va_parser, args=[4], axis=1)
df['duration2']=df.apply(va_parser, args=[5], axis=1)
df.head()

,Unnamed: 0,hadm_id,subject_id,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,temp,bg_temp,hr,spo2,rr,sbp,dbp,meanbp,weight,height,cardiac_index,pt,ptt,inr,inr_1,fibrinogen,hb,hematocrit,plts,wcc,lymphocytes,neutrophils,alp,ast,alt,ggt,bilirubin_indirect,bilirubin_direct,bilirubin_total,chloride,magnesium,potassium,crp,bleed_time,albumin,creatinine,free_calcium,sodium,bicarb,bun,hba1c,glucose,lactate,po2,pco2,baseexcess,ph,aado2,fio2,ffp,insulin,cryo,prbc,infection,ventrate,tidalvol,vent_array,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,admittime,dischtime,intime,outtime,ext_time,reint_time,los,icustay_seq,deathtime,plt,diab_un,diab_cc,dtoutput,specimen,dod,int_time1,ext_time1,duration1,int_time2,ext_time2,duration2
0,0,195663,27328,M,white,SINGLE,Private,ENGL,0,0,0,0,1,"[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",84.0,172.72,NaN,NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 2, 2, 0...","[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,NaN,NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,NaN,NaN,NaN,NaN,NaN,"[{'suspected_infection_time': None, 'antibioti...",NaN,NaN,"[{'starttime': datetime.datetime(2198, 1, 31, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,2198-01-31 08:00:00,2198-02-04 12:00:00,2198-01-31 12:27:58,2198-02-02 19:06:39,2198-01-31 22:00:00,NaT,2.2769,1,NaT,"[{'charttime': datetime.datetime(2198, 1, 31, ...",0,0,NaN,NaN,NaT,2198-01-31 17:00:00,2198-01-31 22:00:00,5.000000,NaT,NaT,NaN
1,1,106984,6280,F,other,DIVORCED,Self Pay,SPAN,1,0,0,0,0,"[{'charttime': datetime.datetime(2198, 5, 8, 1...",NaN,"[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...",60.0,170.18,"[{'charttime': datetime.datetime(2198, 5, 8, 1...",NaN,"[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...",NaN,"[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 10, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2198, 5, 9, 3...","[{'charttime': datetime.datetime(2198, 5, 9, 3...","[{'charttime': datetime.datetime(2198, 5, 8, 1...

In [289]:
def infection_parser(value, timelimit):
    if value == np.NaN:
        return np.NaN
    else:
        a = value
        a = a.replace('\n ','')
        a = a.replace('[','')
        a = a.replace(']','')
        a = a.replace("{'charttime': datetime.datetime",'')
        split = a.split('}')

In [290]:
# df_infection

In [291]:
# def ts_parser(value, timelimit):
#     """
#     Takes single string of timeseries data in MIMIC format and returns the mean, max, min values   
#     Parameters
#     ----------
#     value : single string of timeseries data in MIMIC format
#     timelimit : time (in hours) from the first data entry to include data up to

#     Returns
#     -------
#     avg : mean of all values within specified time period
#     max_: maximum of all values within specified time period
#     min_: minimum of all values within specified time period
#     """
#     if value == np.NaN:
#         return np.NaN, np.NaN, np.NaN
#     else:
#         a = value
#         a = a.replace('\n ','')
#         a = a.replace('[','')
#         a = a.replace(']','')
#         a = a.replace("{'charttime': datetime.datetime",'')
#         split = a.split('}')
#         del split[-1]
#         times = []
#         values = []
#         for n in range(0,len(split)):
#             subsplit = split[n].split(", 'value'")
#             t = datetime.strptime(subsplit[0],'(%Y, %m, %d, %H, %M)')
#             times.append(t)
#             v = float(subsplit[1].replace(': ',''))
#             values.append(v)
#         starttime = times[0]
#         endtime = times[0] + timedelta(hours=timelimit)
#         #find the average
#         incl_values = []
#         for n in range(0,len(split)):
#             if times[n] > starttime and times[n] < endtime: 
#                 incl_values.append(values[n])
#         print(incl_values)
#         avg = statistics.mean(incl_values)
#         max_ = max(incl_values)
#         min_ = min(incl_values)
#         return avg, max_, min_

def ts_parser2(value, timeDelta=None, timeLimits=None, valuename='value'):
    # timeDelta is timedelta in hours from earliest entry
    # timeLimits = (startTime, endTime)
    # if both timeDelta and timeLimits are provided, timeDelta overrules.
    # if both are None, then all timepoints are accepted
    
    if value == np.NaN or pd.isna(value):
        return np.NaN, np.NaN, np.NaN
    
    a = value.replace("'", '"')
    a = a.replace('\n ...\n',',').replace('\n', ',').replace('...', '')
    a = a.replace('datetime.', '"dt.')
    a = a.replace(f'), "{valuename}"', f')", "{valuename}"')
    a = a.replace('"unit": None', '"unit": "None"')
    a = a.replace('starttime', 'charttime')
    a = json.loads(a)
    b = [(eval(i['charttime']), i[valuename]) for i in a]
    
    if timeDelta:
        startTime = min(b, key=lambda x:x[0])[0]
        inc_b = [i[1] for i in b if i[0] <= startTime + dt.timedelta(hours=timeDelta)]
    else:
        if timeLimits:
            inc_b = [i[1] for i in b if i[0] >= timeLimits[0] and i[0] <= timeLimits[1]]
        else:
            inc_b = [i[1] for i in b]
    if len(inc_b) == 0:
        return np.NaN, np.NaN, np.NaN
    
    return sum(inc_b) / len(inc_b), max(inc_b), min(inc_b)

# test_x = df[timeseries].iloc[0,0]
# print(ts_parser(test_x,12))
# print(ts_parser2(test_x, timeDelta=12))
# print()
# test_y = df['bg_temp'][9]
# print(test_y)
# print('Parser1: ', ts_parser(test_y, 36))
# print('Parser2: ', ts_parser2(test_y, timeDelta=36))

### 1.3: Handling missing data

In [292]:
# delete all rows where int_time1 OR ext_time1 are missing
df = df[~(pd.isnull(df['int_time1']) | pd.isnull(df['ext_time1']))]
df = df.reset_index()
df = df.drop(axis=1, columns=['index', 'Unnamed: 0'], inplace=False)
df

,hadm_id,subject_id,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,temp,bg_temp,hr,spo2,rr,sbp,dbp,meanbp,weight,height,cardiac_index,pt,ptt,inr,inr_1,fibrinogen,hb,hematocrit,plts,wcc,lymphocytes,neutrophils,alp,ast,alt,ggt,bilirubin_indirect,bilirubin_direct,bilirubin_total,chloride,magnesium,potassium,crp,bleed_time,albumin,creatinine,free_calcium,sodium,bicarb,bun,hba1c,glucose,lactate,po2,pco2,baseexcess,ph,aado2,fio2,ffp,insulin,cryo,prbc,infection,ventrate,tidalvol,vent_array,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,admittime,dischtime,intime,outtime,ext_time,reint_time,los,icustay_seq,deathtime,plt,diab_un,diab_cc,dtoutput,specimen,dod,int_time1,ext_time1,duration1,int_time2,ext_time2,duration2
0,195663,27328,M,white,SINGLE,Private,ENGL,0,0,0,0,1,"[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",84.00,172.72,NaN,NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 2, 2, 0...","[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,NaN,NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,"[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...","[{'charttime': datetime.datetime(2198, 1, 31, ...",NaN,NaN,NaN,NaN,NaN,NaN,"[{'suspected_infection_time': None, 'antibioti...",NaN,NaN,"[{'starttime': datetime.datetime(2198, 1, 31, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,2198-01-31 08:00:00,2198-02-04 12:00:00,2198-01-31 12:27:58,2198-02-02 19:06:39,2198-01-31 22:00:00,NaT,2.276900,1,NaT,"[{'charttime': datetime.datetime(2198, 1, 31, ...",0,0,NaN,NaN,NaT,2198-01-31 17:00:00,2198-01-31 22:00:00,5.000000,NaT,NaT,NaN
1,106984,6280,F,other,DIVORCED,Self Pay,SPAN,1,0,0,0,0,"[{'charttime': datetime.datetime(2198, 5, 8, 1...",NaN,"[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...",60.00,170.18,"[{'charttime': datetime.datetime(2198, 5, 8, 1...",NaN,"[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...",NaN,"[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 8, 1...","[{'charttime': datetime.datetime(2198, 5, 10, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'charttime': datetime.datetime(2198, 5, 9, 3...","[{'charttime': datetime.datetime(2198, 5, 9, 3...","[{'charttime': datetime.datetime(2198, 5, 8, 1...",NaN,NaN,N

#### 1.3.0 Assessing for missing data

In [293]:
# formula for checking % missing values
def missing_values_table(df): 
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0: 'Missing Values', 1: '% Missing Values'})
    return mis_val_table_ren_columns

missing_data = missing_values_table(df)

In [294]:
#set limit and get list of variables missing above limit in `missing_cols`
missing_limit = 50
missing_cols = missing_data.loc[missing_data['% Missing Values']>missing_limit].index.tolist()
print(missing_cols)
missing_data = missing_data.loc[missing_data['% Missing Values']>missing_limit]
missing_data = missing_data.sort_values(by=['% Missing Values'])
pd.set_option('display.max_rows', None)
missing_data

['bg_temp', 'cardiac_index', 'plts', 'lymphocytes', 'neutrophils', 'alp', 'ast', 'alt', 'ggt', 'bilirubin_indirect', 'bilirubin_direct', 'bilirubin_total', 'crp', 'bleed_time', 'albumin', 'aado2', 'fio2', 'ffp', 'cryo', 'ventrate', 'tidalvol', 'reint_time', 'deathtime', 'dtoutput', 'specimen', 'dod', 'int_time2', 'ext_time2', 'duration2']


,Missing Values,% Missing Values
lymphocytes,5425,57.261980
neutrophils,5426,57.272535
cardiac_index,5940,62.697910
fio2,6463,68.218282
bg_temp,6787,71.638168
bilirubin_total,7320,77.264091
ast,7324,77.306312
alt,7331,77.380198
tidalvol,7360,77.686299
alp,7364,77.728520


In [295]:
pd.reset_option('display.max_rows')

In [225]:
# missing_data.loc[time_cols,:]

In [296]:
df = df.drop(axis=1, columns=list(missing_data.index), inplace=False)
print(list(df.columns))

['hadm_id', 'subject_id', 'gender', 'ethnicity', 'marital_status', 'insurance', 'language', 'aortic', 'mit', 'tricuspid', 'pulmonary', 'cabg', 'temp', 'hr', 'spo2', 'rr', 'sbp', 'dbp', 'meanbp', 'weight', 'height', 'pt', 'ptt', 'inr', 'inr_1', 'fibrinogen', 'hb', 'hematocrit', 'wcc', 'chloride', 'magnesium', 'potassium', 'creatinine', 'free_calcium', 'sodium', 'bicarb', 'bun', 'hba1c', 'glucose', 'lactate', 'po2', 'pco2', 'baseexcess', 'ph', 'insulin', 'prbc', 'infection', 'vent_array', 'reintubation', 'liver_severe', 'liver_mild', 'rheum', 'cvd', 'aids', 'ckd', 'copd', 'arrhythmia', 'pud', 'smoking', 'pvd', 'paraplegia', 'ccf', 'met_ca', 't2dm', 't1dm', 'malig', 'mi', 'dementia', 'first_careunit', 'last_careunit', 'admission_location', 'admission_type', 'hospital_expire_flag', 'admittime', 'dischtime', 'intime', 'outtime', 'ext_time', 'los', 'icustay_seq', 'plt', 'diab_un', 'diab_cc', 'int_time1', 'ext_time1', 'duration1']


In [227]:
# option 2: impute data based on median


In [228]:
# option 3: multiple imputation

# x = missing_data.loc[missing_data['% Missing Values']> 0]
# x.loc[[i for i in x.index if i not in time_cols],:]

#### 1.3.1 Creating summary fields for time-series data

In [229]:
# checking that ts_parser2() works for the timeseries columns

# for j in timeseries:
#     for i in range(len(df[j])):
#         try:
#             if j in timeseries_valuenames:
#                 ts_parser2(df[j][i], timeDelta=36, valuename=timeseries_valuenames[j])
#             else:
#                 ts_parser2(df[j][i], timeDelta=36)
#         except
#             print(j, i)
#             break
#     print(j, 'Fine')

#### 1.3.2 Beginning imputation

In [297]:
df['int_time1']

0      2198-01-31 17:00:00
1      2198-05-08 17:00:00
2      2189-02-18 13:00:00
3      2118-01-25 13:00:00
4      2198-01-01 13:00:00
               ...        
9469   2121-05-02 12:00:00
9470   2111-11-19 19:00:00
9471   2156-03-01 16:00:00
9472   2127-03-05 12:00:00
9473   2193-07-03 16:00:00
Name: int_time1, Length: 9474, dtype: datetime64[ns]

In [300]:
dfForImpute = pd.DataFrame([0 for i in range(df.shape[0])])

# generating timeseries summary values
for column in timeseries:
    if column not in df.columns:
        continue
    x = timeseries_valuenames[column] if column in timeseries_valuenames else "value"
    meanList = []
    maxList = []
    minList = []
    for i in range(len(df[column])):
        y = ts_parser2(df[column][i], timeLimits=(df['int_time1'][i].to_pydatetime(), df['ext_time1'][i].to_pydatetime()), valuename=x)
        meanList.append(y[0])
        maxList.append(y[1])
        minList.append(y[2])
    dfForImpute[column+'_mean'] = meanList
    dfForImpute[column+'_max'] = maxList
    dfForImpute[column+'_min'] = minList

In [301]:
# add on non-time data for imputation
extraColumns = [i for i in df.columns if i not in list(dfForImpute.columns) + timeseries + ['infection', 'vent_array', 'int_time1', 'ext_time1'] + ptinfo + adm_num]
for i in extraColumns:
    if i in ('weight', 'height', 'duration1'):
        dfForImpute[i] = df[i]
    else:
        dfForImpute[i] = df[i].astype('category')
dfForImpute = dfForImpute.copy()

C:\Users\zhepu\AppData\Local\Temp/ipykernel_53276/411917084.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dfForImpute[i] = df[i].astype('category')
C:\Users\zhepu\AppData\Local\Temp/ipykernel_53276/411917084.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dfForImpute[i] = df[i]


In [302]:
# before imputation again
dfForImpute

,0,temp_mean,temp_max,temp_min,hr_mean,hr_max,hr_min,spo2_mean,spo2_max,spo2_min,rr_mean,rr_max,rr_min,sbp_mean,sbp_max,sbp_min,dbp_mean,dbp_max,dbp_min,meanbp_mean,meanbp_max,meanbp_min,pt_mean,pt_max,pt_min,ptt_mean,ptt_max,ptt_min,inr_mean,inr_max,inr_min,inr_1_mean,inr_1_max,inr_1_min,fibrinogen_mean,fibrinogen_max,fibrinogen_min,hb_mean,hb_max,hb_min,hematocrit_mean,hematocrit_max,hematocrit_min,wcc_mean,wcc_max,wcc_min,chloride_mean,chloride_max,chloride_min,magnesium_mean,magnesium_max,magnesium_min,potassium_mean,potassium_max,potassium_min,creatinine_mean,creatinine_max,creatinine_min,free_calcium_mean,free_calcium_max,free_calcium_min,sodium_mean,sodium_max,sodium_min,bicarb_mean,bicarb_max,bicarb_min,bun_mean,bun_max,bun_min,hba1c_mean,hba1c_max,hba1c_min,glucose_mean,glucose_max,glucose_min,lactate_mean,lactate_max,lactate_min,po2_mean,po2_max,po2_min,pco2_mean,pco2_max,pco2_min,baseexcess_mean,baseexcess_max,baseexcess_min,ph_mean,ph_max,ph_min,insulin_mean,insulin_max,insulin_min,prbc_mean,prbc_max,prbc_min,plt_mean,plt_max,plt_min,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,weight,height,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,diab_un,diab_cc,duration1
0,0,37.314816,37.444445,37.111113,88.307692,107.0,63.0,99.818182,100.0,98.0,17.714286,34.0,14.0,126.384615,161.0,108.0,59.230769,69.0,50.0,78.153846,86.0,68.0,NaN,NaN,NaN,29.8,29.8,29.8,1.2,1.2,1.2,1.2,1.2,1.2,NaN,NaN,NaN,12.100,12.1,12.1,NaN,NaN,NaN,14.70,14.7,14.7,107.000000,107.0,107.0,NaN,NaN,NaN,3.700000,3.7,3.7,0.80,0.8,0.8,1.150000,1.15,1.15,140.000000,140.0,140.0,25.0,25.0,25.0,13.0,13.0,13.0,NaN,NaN,NaN,140.000000,168.0,112.0,NaN,NaN,NaN,1.000000,1.0,1.0,42.000000,42.0,42.0,1.000000,1.0,1.0,7.400000,7.40,7.40,NaN,NaN,NaN,NaN,NaN,NaN,255.000000,255.0,255.0,M,white,SINGLE,Private,ENGL,0,0,0,0,1,84.00,172.72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,0,0,5.000000
1,0,37.071429,38.500000,35.200001,92.827586,119.0,78.0,97.590164,100.0,65.0,11.873016,21.0,7.5,95.879310,134.0,75.0,51.879310,75.0,35.0,66.086207,101.0,45.0,NaN,NaN,NaN,42.8,47.5,38.1,1.6,1.8,1.4,1.6,1.8,1.4,NaN,NaN,NaN,7.075,10.0,6.0,21.25,30.0,18.0,NaN,NaN,NaN,108.000000,108.0,108.0,2.1,2.1,2.1,4.366667,4.8,3.6,0.60,0.6,0.6,1.135000,1.21,1.00,142.000000,142.0,142.0,26.0,26.0,26.0,12.0,12.0,12.0,NaN,NaN,NaN,146.928571,195.0,118.0,NaN,NaN,NaN,0.555556,2.0,-1.0,43.888889,50.0,36.0,0.555556,2.0,-1.0,7.387778,7.43,7.35,262.500000,314.000000,211.000000,375.0,375.0,375.0,122.500000,141.0,104.0,F,other,DIVORCED,Self Pay,SPAN,1,0,0,0,0,60.00,170.18,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,0,0,16.483333
2,0,36.851515,37.799999,35.400002,86.000000,88.0,70.0,99.965517,100.0,99.0,14.764706,25.0,9.0,119.060606,170.0,89.0,59.000000,88.0,45.0,79.787879,121.0,57.0,NaN,NaN,NaN,59.3,59.3,59.3,1.6,1.6,1.6,1.6,1.6,1.6,NaN,NaN,NaN,10.060,11.9,6.5,28.00,35.0,20.0,21.85,22.8,20.9,111.000000,113.0,109.0,2.6,2.6,2.6,4.285714,4.9,3.1,1.20,1.2,1.2,1.156000,1.32,0.95,140.333333,143.0,139.0,26.5,28.0,25.0,19.0,19.0,19.0,NaN,NaN,NaN,119.080000,188.0,92.0,NaN,NaN,NaN,1.166667,3.0,-1.0,41.333333,48.0,34.0,1.166667,3.0,-1.0,7.404286,7.48,7.32,NaN,NaN,NaN,375.0,375.0,375.0,144.500000,148.0,141.0,F,NaN,MARRIED,Medicare,NaN,1,0,0,0,1,57.00,165.10,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,0,0,20.000000
3,0,37.286667,38.700001,36.700001,93.333333,104.0,80.0,98.571429,100.0,94.0,16.857143,21.0,12.0,118.333333,162.0,85.0,58.600000,76.0,40.0,75.800000,99.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.000000,106.0,106.0,1.8,1.8,1.8,3.650000,3.7,3.6,0.40,0.4,0.4,1.115000,1.13,1.10,138.000000,138.0,138.0,27.0,27.0,27.0,14.0,14.0,14.0,NaN,NaN,NaN,140.

In [303]:
dfForImpute2 = dfForImpute[dfForImpute.columns[:]]

kds = mf.ImputationKernel(
  dfForImpute2,
  datasets=1,
  save_all_iterations=True,
  random_state=1991
)

# Run the MICE algorithm for 3 iterations
kds.mice(3)

print(kds)

dfImputed = kds.complete_data(dataset=0, inplace=False)
print(dfImputed.isnull().sum(0))

# after imputation
dfImputed.head(10)

              Class: ImputationKernel
           Datasets: 1
         Iterations: 3
  Imputed Variables: 105
save_all_iterations: True
0                       0
temp_mean               0
temp_max                0
temp_min                0
hr_mean                 0
                       ..
admission_type          0
hospital_expire_flag    0
diab_un                 0
diab_cc                 0
duration1               0
Length: 140, dtype: int64


,0,temp_mean,temp_max,temp_min,hr_mean,hr_max,hr_min,spo2_mean,spo2_max,spo2_min,rr_mean,rr_max,rr_min,sbp_mean,sbp_max,sbp_min,dbp_mean,dbp_max,dbp_min,meanbp_mean,meanbp_max,meanbp_min,pt_mean,pt_max,pt_min,ptt_mean,ptt_max,ptt_min,inr_mean,inr_max,inr_min,inr_1_mean,inr_1_max,inr_1_min,fibrinogen_mean,fibrinogen_max,fibrinogen_min,hb_mean,hb_max,hb_min,hematocrit_mean,hematocrit_max,hematocrit_min,wcc_mean,wcc_max,wcc_min,chloride_mean,chloride_max,chloride_min,magnesium_mean,magnesium_max,magnesium_min,potassium_mean,potassium_max,potassium_min,creatinine_mean,creatinine_max,creatinine_min,free_calcium_mean,free_calcium_max,free_calcium_min,sodium_mean,sodium_max,sodium_min,bicarb_mean,bicarb_max,bicarb_min,bun_mean,bun_max,bun_min,hba1c_mean,hba1c_max,hba1c_min,glucose_mean,glucose_max,glucose_min,lactate_mean,lactate_max,lactate_min,po2_mean,po2_max,po2_min,pco2_mean,pco2_max,pco2_min,baseexcess_mean,baseexcess_max,baseexcess_min,ph_mean,ph_max,ph_min,insulin_mean,insulin_max,insulin_min,prbc_mean,prbc_max,prbc_min,plt_mean,plt_max,plt_min,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,weight,height,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,diab_un,diab_cc,duration1
0,0,37.314816,37.444445,37.111113,88.307692,107.0,63.0,99.818182,100.0,98.0,17.714286,34.0,14.0,126.384615,161.0,108.0,59.230769,69.0,50.0,78.153846,86.0,68.0,12.857143,13.1,12.8,29.80,29.8,29.8,1.2000,1.2,1.2,1.200000,1.2,1.2,229.0,256.0,211.0,12.100000,12.1,12.1,36.500000,36.0,36.0,14.70,14.7,14.7,107.0,107.0,107.0,2.500,2.6,2.4,3.700000,3.7,3.7,0.800000,0.8,0.8,1.150000,1.15,1.15,140.000000,140.0,140.0,25.0,25.0,25.0,13.0,13.0,13.0,5.5,5.5,5.5,140.000000,168.0,112.0,1.433333,2.0,1.1,1.000000,1.0,1.0,42.000000,42.0,42.0,1.000000,1.0,1.0,7.400000,7.40,7.40,185.666667,252.000000,0.0,375.000000,375.000000,375.000000,255.0,255.0,255.0,M,white,SINGLE,Private,ENGL,0,0,0,0,1,84.0,172.72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,0,0,5.000000
1,0,37.071429,38.500000,35.200001,92.827586,119.0,78.0,97.590164,100.0,65.0,11.873016,21.0,7.5,95.879310,134.0,75.0,51.879310,75.0,35.0,66.086207,101.0,45.0,17.300000,19.8,15.7,42.80,47.5,38.1,1.6000,1.8,1.4,1.600000,1.8,1.4,179.0,213.0,155.0,7.075000,10.0,6.0,21.250000,30.0,18.0,16.05,21.3,11.6,108.0,108.0,108.0,2.100,2.1,2.1,4.366667,4.8,3.6,0.600000,0.6,0.6,1.135000,1.21,1.00,142.000000,142.0,142.0,26.0,26.0,26.0,12.0,12.0,12.0,5.5,5.5,5.5,146.928571,195.0,118.0,2.166667,2.7,1.5,0.555556,2.0,-1.0,43.888889,50.0,36.0,0.555556,2.0,-1.0,7.387778,7.43,7.35,262.500000,314.000000,211.0,375.000000,375.000000,375.000000,122.5,141.0,104.0,F,other,DIVORCED,Self Pay,SPAN,1,0,0,0,0,60.0,170.18,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,0,0,16.483333
2,0,36.851515,37.799999,35.400002,86.000000,88.0,70.0,99.965517,100.0,99.0,14.764706,25.0,9.0,119.060606,170.0,89.0,59.000000,88.0,45.0,79.787879,121.0,57.0,17.500000,17.6,17.5,59.30,59.3,59.3,1.6000,1.6,1.6,1.600000,1.6,1.6,260.5,301.0,238.0,10.060000,11.9,6.5,28.000000,35.0,20.0,21.85,22.8,20.9,111.0,113.0,109.0,2.600,2.6,2.6,4.285714,4.9,3.1,1.200000,1.2,1.2,1.156000,1.32,0.95,140.333333,143.0,139.0,26.5,28.0,25.0,19.0,19.0,19.0,5.6,5.6,5.6,119.080000,188.0,92.0,1.400000,1.6,1.2,1.166667,3.0,-1.0,41.333333,48.0,34.0,1.166667,3.0,-1.0,7.404286,7.48,7.32,190.333333,330.000000,0.5,375.000000,375.000000,375.000000,144.5,148.0,141.0,F,white,MARRIED,Medicare,CANT,1,0,0,0,1,57.0,165.10,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,0,0,20.000000
3,0,37.286667,38.700001,36.700001,93.333333,104.0,80.0,98.571429,100.0,94.0,16.857143,21.0,12.0,118.333333,162.0,85.0,58.600000,76.0,40.0,75.800000,99.0,56.0,16.450000,17.6,11.6,25.50,25.6,25.3,1.5500,1.6,1.0,1.544444,1.6,1.0,135.0,

In [306]:
y = np.asarray([i.to_pydatetime() for i in df["int_time1"]])
z = np.asarray([i.to_pydatetime() for i in df["outtime"]])
dfImputed['icu_stay_duration'] = [i.total_seconds() for i in z-y]
dfImputed[['hadm_id','subject_id']] = df[['hadm_id','subject_id']]
dfImputed

,0,temp_mean,temp_max,temp_min,hr_mean,hr_max,hr_min,spo2_mean,spo2_max,spo2_min,rr_mean,rr_max,rr_min,sbp_mean,sbp_max,sbp_min,dbp_mean,dbp_max,dbp_min,meanbp_mean,meanbp_max,meanbp_min,pt_mean,pt_max,pt_min,ptt_mean,ptt_max,ptt_min,inr_mean,inr_max,inr_min,inr_1_mean,inr_1_max,inr_1_min,fibrinogen_mean,fibrinogen_max,fibrinogen_min,hb_mean,hb_max,hb_min,hematocrit_mean,hematocrit_max,hematocrit_min,wcc_mean,wcc_max,wcc_min,chloride_mean,chloride_max,chloride_min,magnesium_mean,magnesium_max,magnesium_min,potassium_mean,potassium_max,potassium_min,creatinine_mean,creatinine_max,creatinine_min,free_calcium_mean,free_calcium_max,free_calcium_min,sodium_mean,sodium_max,sodium_min,bicarb_mean,bicarb_max,bicarb_min,bun_mean,bun_max,bun_min,hba1c_mean,hba1c_max,hba1c_min,glucose_mean,glucose_max,glucose_min,lactate_mean,lactate_max,lactate_min,po2_mean,po2_max,po2_min,pco2_mean,pco2_max,pco2_min,baseexcess_mean,baseexcess_max,baseexcess_min,ph_mean,ph_max,ph_min,insulin_mean,insulin_max,insulin_min,prbc_mean,prbc_max,prbc_min,plt_mean,plt_max,plt_min,gender,ethnicity,marital_status,insurance,language,aortic,mit,tricuspid,pulmonary,cabg,weight,height,reintubation,liver_severe,liver_mild,rheum,cvd,aids,ckd,copd,arrhythmia,pud,smoking,pvd,paraplegia,ccf,met_ca,t2dm,t1dm,malig,mi,dementia,first_careunit,last_careunit,admission_location,admission_type,hospital_expire_flag,diab_un,diab_cc,duration1,icu_stay_duration,hadm_id,subject_id
0,0,37.314816,37.444445,37.111113,88.307692,107.0,63.0,99.818182,100.0,98.0,17.714286,34.0,14.0,126.384615,161.0,108.0,59.230769,69.0,50.0,78.153846,86.0,68.0,12.857143,13.1,12.8,29.80,29.8,29.8,1.20,1.2,1.2,1.200000,1.2,1.2,229.0,256.0,211.0,12.100,12.1,12.1,36.500000,36.0,36.0,14.70,14.7,14.7,107.000000,107.0,107.0,2.500000,2.6,2.4,3.700000,3.7,3.7,0.800000,0.8,0.8,1.150000,1.15,1.15,140.000000,140.0,140.0,25.0,25.0,25.0,13.0,13.0,13.0,5.5,5.5,5.5,140.000000,168.0,112.0,1.433333,2.0,1.1,1.000000,1.0,1.0,42.000000,42.0,42.0,1.000000,1.0,1.0,7.400000,7.40,7.40,185.666667,252.000000,0.000000,375.000000,375.000000,375.000000,255.000000,255.0,255.0,M,white,SINGLE,Private,ENGL,0,0,0,0,1,84.00,172.72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,0,0,5.000000,180399.0,195663,27328
1,0,37.071429,38.500000,35.200001,92.827586,119.0,78.0,97.590164,100.0,65.0,11.873016,21.0,7.5,95.879310,134.0,75.0,51.879310,75.0,35.0,66.086207,101.0,45.0,17.300000,19.8,15.7,42.80,47.5,38.1,1.60,1.8,1.4,1.600000,1.8,1.4,179.0,213.0,155.0,7.075,10.0,6.0,21.250000,30.0,18.0,16.05,21.3,11.6,108.000000,108.0,108.0,2.100000,2.1,2.1,4.366667,4.8,3.6,0.600000,0.6,0.6,1.135000,1.21,1.00,142.000000,142.0,142.0,26.0,26.0,26.0,12.0,12.0,12.0,5.5,5.5,5.5,146.928571,195.0,118.0,2.166667,2.7,1.5,0.555556,2.0,-1.0,43.888889,50.0,36.0,0.555556,2.0,-1.0,7.387778,7.43,7.35,262.500000,314.000000,211.000000,375.000000,375.000000,375.000000,122.500000,141.0,104.0,F,other,DIVORCED,Self Pay,SPAN,1,0,0,0,0,60.00,170.18,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,0,0,16.483333,182760.0,106984,6280
2,0,36.851515,37.799999,35.400002,86.000000,88.0,70.0,99.965517,100.0,99.0,14.764706,25.0,9.0,119.060606,170.0,89.0,59.000000,88.0,45.0,79.787879,121.0,57.0,17.500000,17.6,17.5,59.30,59.3,59.3,1.60,1.6,1.6,1.600000,1.6,1.6,260.5,301.0,238.0,10.060,11.9,6.5,28.000000,35.0,20.0,21.85,22.8,20.9,111.000000,113.0,109.0,2.600000,2.6,2.6,4.285714,4.9,3.1,1.200000,1.2,1.2,1.156000,1.32,0.95,140.333333,143.0,139.0,26.5,28.0,25.0,19.0,19.0,19.0,5.6,5.6,5.6,119.080000,188.0,92.0,1.400000,1.6,1.2,1.166667,3.0,-1.0,41.333333,48.0,34.0,1.166667,3.0,-1.0,7.404286,7.48,7.32,190.333333,330.000000,0.500000,375.000000,375.000000,375.000000,144.500000,148.0,141.0,F,white,MARRIED,Medicare,CANT,1,0,0,0,1,57.00,165.10,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,CSRU,CSRU,PHYSICIAN REFERRAL,ELECTIVE,0,0,0,20.000000,175068.0,123613,15201
3,0,37.286667,38.700001,36.700001,93.333333,104.0,80.0,98.571429,100.0,94.0,16.857143,21

In [305]:
dfImputed.to_csv('imputed.csv')